In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import random
import os

In [97]:
URL = 'https://github.com/search?q=machine+learning&type=repositories&p=1'
# HEADER = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36', 'Accept-Language':'en-US, en;q=0.5'})

In [98]:
# webpage = requests.get(URL, headers=HEADER)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [99]:
# webpage
driver.get(URL)

In [100]:
# soup = BeautifulSoup(webpage.content, 'html.parser')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="fe7d04e97473f0c59e6aa74f364bd9d9")>

In [101]:
time.sleep(3)

In [102]:
full_repo_name = driver.find_element(By.XPATH, "//a[contains(@href, '/josephmisiti/awesome-machine-learning')]//span").text

In [103]:
full_repo_name

'josephmisiti/awesome-machine-learning'

In [104]:
author, repo_name = full_repo_name.split('/')

In [105]:
author

'josephmisiti'

In [106]:
repo_name

'awesome-machine-learning'

In [112]:
description = driver.find_element(By.CSS_SELECTOR, ".Box-sc-62in7e-0.jsbtiO").text

In [113]:
description

'A curated list of awesome Machine Learning frameworks, libraries and software.'

In [119]:
language = driver.find_element(By.XPATH, "//li//span[contains(@aria-label, 'language')]").text

In [120]:
language

'Python'

In [121]:
stars = driver.find_element(By.XPATH, "//a[contains(@href, '/josephmisiti/awesome-machine-learning/stargazers')]//span").text

In [122]:
stars

'70.5k'

In [123]:
driver.execute_script(f"window.open('{'https://github.com/josephmisiti/awesome-machine-learning'}', '_blank');")
driver.switch_to.window(driver.window_handles[1])
time.sleep(3)

In [124]:
# forks_element = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/forks') or contains(text(), 'Forks')]"))
# )
# forks_element = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/forks')]//span"))
# )
forks_text = driver.execute_script("""
    const el = document.querySelector('a[href*="/forks"] span');
    return el ? el.innerText.trim() : "";
""")

In [125]:
forks_text

'15.1k'

In [127]:
last_updated = driver.find_element(By.TAG_NAME, 'relative-time').get_attribute('datetime')

In [128]:
last_updated

'2025-10-28T17:49:09.000Z'

In [129]:
print(f'Author name: {author}')
print(f'Repo name: {repo_name}')
print(f'Language name: {language}')
print(f'Satrs: {stars}')
print(f'Forks: {forks_text}')
print(f'Last updated: {last_updated}')

Author name: josephmisiti
Repo name: awesome-machine-learning
Language name: Python
Satrs: 70.5k
Forks: 15.1k
Last updated: 2025-10-28T17:49:09.000Z


In [2]:
# Chrome browser open karne ke liye
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
]

options.add_argument(f"user-agent={random.choice(user_agents)}")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

print("✅ Chrome WebDriver started successfully!")

✅ Chrome WebDriver started successfully!


In [3]:
def get_forks(repo_url):
    """Ek single repo ke page me jaake forks count return karta hai."""
    driver.execute_script(f"window.open('{repo_url}', '_blank');")
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    
    try:
            fork = driver.execute_script("""
        const el = document.querySelector('a[href*="/forks"] span');
        return el ? el.innerText.trim() : "";
    """)
    except:
        fork = "0"
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return fork


In [27]:
def scrape_github_repos(keyword, pages=5):
    """GitHub repositories scrape with repo details and forks."""
    data = []
    
    for page in range(91, pages + 1):
        url = f"https://github.com/search?q={keyword}&type=repositories&p={page}"
        print(f"🔍 Scraping page {page}...")
        driver.get(url)
        time.sleep(random.uniform(15, 30))
        
        # Har repo card dhundho
        repo_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'Box-sc-62in7e-0 fXzjPH')]")
        
        for card in repo_cards:
            try:
                repo_name = card.find_element(By.TAG_NAME, "a").text
                repo_link = card.find_element(By.TAG_NAME, "a").get_attribute("href")
                author, repo = repo_name.split('/')
            except:
                continue
            
            # Description
            try:
                desc = card.find_element(By.CSS_SELECTOR, ".Box-sc-62in7e-0.jsbtiO").text
            except:
                desc = ""
            
            # Language
            try:
                lang_elem = card.find_element(By.XPATH, "//li//span[contains(@aria-label, 'language')]")
                lang = lang_elem.get_attribute("aria-label").replace(" language", "")
            except:
                lang = "N/A"
            
            # Stars
            try:
                stars = card.find_element(By.XPATH, ".//a[contains(@href, '/stargazers')]//span").text
            except:
                stars = "0"
            try:
                updated_element = card.find_element(
                    By.XPATH, ".//li[contains(@class, 'Box-sc-62in7e-0') and contains(., 'Updated')]"
                )
                last_updated = updated_element.text.replace("Updated", "").strip()

            except:
                last_updated = "N/A"
            
            # Forks (repo page se)
            fork = get_forks(repo_link)
            
            data.append({
                "Author": author.strip(),
                "Repository": repo.strip(),
                "Description": desc.strip(),
                "Language": lang,
                "Stars": stars,
                "Forks": fork,
                "Repo_URL": repo_link,
                "Last Updated": last_updated
            })
            
            # polite delay
            time.sleep(random.uniform(3, 8))
    
    return pd.DataFrame(data)


In [28]:
df = scrape_github_repos("machine learning", pages=100) 

🔍 Scraping page 91...
🔍 Scraping page 92...
🔍 Scraping page 93...
🔍 Scraping page 94...
🔍 Scraping page 95...
🔍 Scraping page 96...
🔍 Scraping page 97...
🔍 Scraping page 98...
🔍 Scraping page 99...
🔍 Scraping page 100...


In [29]:
filename = "github_data.csv"
if not os.path.exists(filename):
    df.to_csv(filename, index=False)
else:
    df.to_csv(filename, mode='a', header=False, index=False)

In [30]:
data = pd.read_csv(filename)
data.shape

(970, 8)